In [2]:
import numpy as np
from tqdm import tqdm
from mlagents_envs.base_env import ActionTuple
from mlagents_envs.environment import UnityEnvironment
from mlagents_envs.side_channel.engine_configuration_channel import EngineConfigurationChannel

# Random Action

In [3]:
def get_random_action():
    continuous_actions = np.random.uniform(-1, 1, size = 2).reshape(1,2).astype(np.float32)
    discrete_actions = np.array([np.random.randint(2)]).reshape(1,1).astype(np.float32)
    return continuous_actions, discrete_actions

# Sample Action
get_random_action()

(array([[0.83975357, 0.7046182 ]], dtype=float32),
 array([[0.]], dtype=float32))

# Load Environment

In [4]:
# This will open the environment
channel = EngineConfigurationChannel()
env = UnityEnvironment('./mini_football_windows/Mini-Football-Env.exe', side_channels=[channel])

In [5]:
channel.set_configuration_parameters(
    width= 848,
    height= 480,
    quality_level= 5,
    time_scale= 1,
    target_frame_rate= -1,
    capture_frame_rate= 60,
    )

In [6]:
env.step()
behavior_name = list(env.behavior_specs)[0]

def play_episode():
    env.reset()

    decision_steps, terminal_steps = env.get_steps(behavior_name)
    done = False
    step = 0
    reward = 0

    while not done:
        
        step += 1

        state = decision_steps.obs[0][0]
        reward += decision_steps.reward[0]

        actions = get_brain_action(state)
        action = ActionTuple(actions[0], actions[1])
        #action = ActionTuple(get_random_action()[0], get_random_action()[1])
        env.set_actions(behavior_name=behavior_name, action=action)
        env.step()

        decision_steps, terminal_steps = env.get_steps(behavior_name)

        for agent_id in terminal_steps:
            reward += terminal_steps.reward[0]
            done = True

    return reward

In [8]:
env.reset()
all_rewards = []
for i in tqdm(range(100)):
    all_rewards.append(play_episode())

print(np.mean(all_rewards))

100%|██████████| 100/100 [03:40<00:00,  2.20s/it]

0.9694110007055133
